In [1]:
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.crossover.hux import HUX
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize

from generic_optimizer import GenericOptimizer, prepare_splits
from novel_sampling import BiasedBinarySampling, over_sample

from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
import os
import re

from sklearn.preprocessing import LabelEncoder

In [4]:
needs_redo = """0_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
0_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
10_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
11_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
12_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
13_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
14_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
15_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
16_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
17_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
18_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
19_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
1_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
20_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
21_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
22_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
23_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
24_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
25_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
26_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
27_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
28_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
29_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
2_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
30_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
3_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
4_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
5_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
6_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
7_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
8_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error
9_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error"""
needs_redo = needs_redo.split("\n")
print(needs_redo)

['0_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error', '0_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error', '0_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error', '0_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error', '0_regular_sample_bias_population_sonar_calculate_class0_error_calculate_class1_error', '0_regular_sample_bias_population_vehicle0_calculate_class0_error_calculate_class1_error', '0_regular_sample_bias_population_yeast1_calculate_class0_error_calculate_class1_error', '10_regular_sample_bias_population_magic_calculate_class0_error_calculate_class1_error', '10_regular_sample_bias_population_phoneme_calculate_class0_error_calculate_class1_error', '10_regular_sample_bias_population_pima_calculate_class0_error_calculate_class1_error', '10_regular_sample_bias_population_segment0_calculate_class0_error_calculate_class1_error', '10_regular_sam

In [ ]:
configs = []
from generic_optimizer import *

evaluation_schemes = [
	# [GenericOptimizer.calculate_overall_error, GenericOptimizer.calculate_num_examples],
	# [GenericOptimizer.calculate_overall_error, GenericOptimizer.calculate_overall_inverse_f1, GenericOptimizer.calculate_num_examples],
	# [GenericOptimizer.calculate_class0_error, GenericOptimizer.calculate_class1_error],
	[GenericOptimizer.calculate_overall_error]
]

pattern = r'\.([a-zA-Z_][a-zA-Z0-9_]*)\s'

sampling_methods = [
	"over_sample",
	"regular_sample"
]

population_initialization_names = [
    "rand_population",
    "bias_population"
]

for sampling_name in sampling_methods:
	for initial_population in population_initialization_names:
		for objectives in evaluation_schemes:
			objectives_names = [re.search(pattern, str(objective_name)).group(1) for objective_name in objectives]
			for folder in os.listdir('../Datasets'):
				x, y, splits = prepare_splits(os.path.join('../Datasets', folder, f"{folder}.csv"))

				num_iters = 0
				for train_idx, test_idx, validation_idx in splits: # 31 splits for Wilcoxon rank-sum
					
					objectives_id = "_".join(objectives_names)
					run_name = f"{num_iters}_{sampling_name}_{initial_population}_{folder}_{objectives_id}"
					
					config = {
						"Dataset": folder,
						"Over Sample": sampling_name,
						"Initial Population": initial_population,
						"Objective Names": objectives_names,
						"Iter Number": num_iters,
						"Run Name": run_name,
						"Executable Objectives": objectives,
						"x": x, "y": y,
						"train_idx": train_idx,
						"validation_idx": validation_idx,
						"test_idx": test_idx,
					}
					num_iters += 1
					# if run_name in needs_redo:
					configs.append(config)

print(f"Created {len(configs)} executions")

Created 620 executions


In [8]:
def execute(execution_config):
	
	if os.path.exists(f"optimizations/{execution_config['Run Name']}.pickle"):
		return 
	
	train_idx = execution_config["train_idx"]
	validation_idx = execution_config["validation_idx"]
	x_train, y_train = execution_config["x"][train_idx], execution_config["y"][train_idx]
	x_validation, y_validation = execution_config["x"][validation_idx], execution_config["y"][validation_idx]
	
	if execution_config["Over Sample"] == "over_sample":
		x_train, y_train = over_sample(x_train, y_train)
	
	objectives = execution_config["Executable Objectives"]

	problem = GenericOptimizer(
		x_train, 
		y_train, 
		x_validation, 
		y_validation,
		objectives,
		"Sequential"
	)

	initial_population = BinaryRandomSampling() if execution_config["Initial Population"] == "rand_population" else BiasedBinarySampling(y_train, 0.5, 0.7)

	algorithm = NSGA2(
		pop_size=GenericOptimizer.population_size, 
		sampling=initial_population, 
		crossover=HUX(), 
		mutation=BitflipMutation(), 
		eliminate_duplicates=True)
	
	result = minimize(
		problem, 
		algorithm, 
		('n_gen', GenericOptimizer.population_size)
	)

	execution_config['Result'] = result

	with open(f"optimizations/{execution_config['Run Name']}.pickle", 'wb') as fh:
		pickle.dump(execution_config, fh, pickle.HIGHEST_PROTOCOL)

# for config in configs:
# 	execute(config)
# 	break
	
	
returns = Parallel(n_jobs=-1)(delayed(execute)((config)) for config in configs)
print(f"Sucessfully completed {len(returns)} optimization runs")

Sucessfully completed 620 optimization runs
